In [5]:
!pip install tensorflow==2.4.0 scikit-learn==0.24.1 import_ipynb==0.1.3

ERROR: Could not find a version that satisfies the requirement import_ipynb==0.1.3
ERROR: No matching distribution found for import_ipynb==0.1.3
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [1]:
import tensorflow as tf
from tensorflow import keras
import import_ipynb
from codait_utils import *

importing Jupyter notebook from codait_utils.ipynb


In [ ]:
unzip('.','model.zip')
unzip('.','data.zip')

In [2]:
model = keras.models.load_model('model')

In [3]:
batch_size = 1
img_height = 244
img_width = 244
input_shape = (img_width,img_height)
num_classes=2

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))


Found 97698 files belonging to 2 classes.
Using 78159 files for training.
Found 97698 files belonging to 2 classes.
Using 19539 files for validation.


In [4]:
classes = model.predict(val_ds, batch_size=batch_size)

In [6]:
len(classes)

19539

In [7]:
import numpy as np
y_test = None
for features, labels in val_ds:
    if y_test is not None:
        y_test = np.vstack((y_test,labels.numpy()))
    else:
        y_test = labels.numpy()


In [8]:
from sklearn import metrics
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), classes.argmax(axis=1))

In [9]:
import pandas as pd
pd.DataFrame(matrix)

,0,1
0,17479,33
1,2025,2


In [10]:
import pandas as pd
df = pd.read_csv('fairface_label_val.csv')

In [11]:
from PIL import Image
from numpy import asarray

dataset = []

for i, row in df.iterrows():
    image = 'data/val/'+row['gender'].lower()+'/'+row['file'].split('/')[1] 
    image = Image.open(image)
    image = image.resize((244,244), Image.ANTIALIAS)
    data = asarray(image)
    data.shape = (1, 244, 244, 3)
    label = 0 if row['gender']=="Female" else 1
    prediction = model.predict(data)
    prediction = tf.math.argmax(prediction,1).numpy()[0] #de-onehotencode
    dataset.append([label,prediction])

In [12]:
import numpy as np
dataset = np.array(dataset)

In [13]:
import pandas as pd
df = pd.read_csv('fairface_label_val.csv')

In [14]:
df['labels'] = dataset[:,0].tolist()
df['prediction'] = dataset[:,1].tolist()

In [15]:
df["age_index"] = df.age.apply(lambda x: list(df.age.unique()).index(x))
df["gender_index"] = df.gender.apply(lambda x: list(df.gender.unique()).index(x))
df["race_index"] = df.race.apply(lambda x: list(df.race.unique()).index(x))
df["missclassified"] = df.apply(lambda d: 1 if d.labels!=d.prediction else 0, axis=1)

In [16]:
df

,file,age,gender,race,service_test,labels,prediction,age_index,gender_index,race_index,missclassified
0,val/1.jpg,3-9,Male,East Asian,False,1,0,0,0,0,1
1,val/2.jpg,50-59,Female,East Asian,True,0,0,1,1,0,0
2,val/3.jpg,30-39,Male,White,True,1,0,2,0,1,1
3,val/4.jpg,20-29,Female,Latino_Hispanic,True,0,0,3,1,2,0
4,val/5.jpg,20-29,Male,Southeast Asian,False,1,0,3,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...
10949,val/10950.jpg,30-39,Male,White,True,1,0,2,0,1,1
10950,val/10951.jpg,50-59,Male,White,False,1,0,1,0,1,1
10951,val/10952.jpg,60-69,Male,Latino_Hispanic,False,1,0,7,0,2,1
10952,val/10953.jpg,20-29,Female,East Asian,False,0,1,3,1,0,1


In [26]:
newdf = df.query('race == "White" & age == "20-29"')
matrix = metrics.confusion_matrix(newdf.labels, newdf.prediction)
pd.DataFrame(matrix)

,0,1
0,357,3
1,279,2


In [24]:
newdf = df.query('race == "Latino_Hispanic"')
matrix = metrics.confusion_matrix(newdf.labels, newdf.prediction)
pd.DataFrame(matrix)

,0,1
0,828,2
1,789,4
